In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# 데이터 로드
file_path = './campingData.csv'
camping_data = pd.read_csv(file_path, encoding='utf-8')

# NaN 값을 빈 문자열로 대체
camping_data['lctCl'] = camping_data['lctCl'].fillna('')
camping_data['sbrsCl'] = camping_data['sbrsCl'].fillna('')
camping_data['siteBottomCl1'] = camping_data['siteBottomCl1'].fillna('')
camping_data['siteBottomCl2'] = camping_data['siteBottomCl2'].fillna('')
camping_data['siteBottomCl3'] = camping_data['siteBottomCl3'].fillna('')
camping_data['siteBottomCl4'] = camping_data['siteBottomCl4'].fillna('')
camping_data['siteBottomCl5'] = camping_data['siteBottomCl5'].fillna('')

# MultiLabelBinarizer 초기화
mlb = MultiLabelBinarizer()

# 입지 구분 벡터화
lctCl_vectors = mlb.fit_transform(camping_data['lctCl'].str.split(','))

# 주요 시설 벡터화
sbrsCl_vectors = mlb.fit_transform(camping_data['sbrsCl'].str.split(','))

# 바닥 형태 벡터화
siteBottom_vectors = camping_data[['siteBottomCl1', 'siteBottomCl2', 'siteBottomCl3', 'siteBottomCl4', 'siteBottomCl5']].values

# 모든 특성 결합
feature_vectors = pd.concat([
    pd.DataFrame(lctCl_vectors, index=camping_data.index),
    pd.DataFrame(sbrsCl_vectors, index=camping_data.index),
    pd.DataFrame(siteBottom_vectors, index=camping_data.index)
], axis=1)

# 코사인 유사도 계산
cosine_sim = cosine_similarity(feature_vectors)
cosine_sim_df = pd.DataFrame(cosine_sim, index=camping_data['facltNm'], columns=camping_data['facltNm'])

# 특정 캠핑장과 유사한 캠핑장 추천
def recommend_similar_camps(camp_name, top_n=5):
    similar_camps = cosine_sim_df[camp_name].sort_values(ascending=False).head(top_n + 1).index[1:]
    return similar_camps

# 가상 평점 생성
camping_data['user_id'] = camping_data.index % 10  # 가상 사용자 ID 생성
camping_data['rating'] = 5  # 가상 평점 (모든 캠핑장에 5점)

# Surprise 데이터 준비
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(camping_data[['user_id', 'facltNm', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

# Surprise 모델 학습
algo = SVD()
algo.fit(trainset)

# 예측 및 추천
def recommend_with_surprise(user_id, top_n=5):
    all_camps = camping_data['facltNm'].unique()
    user_ratings = [(user_id, camp, algo.predict(user_id, camp).est) for camp in all_camps]
    top_recommendations = sorted(user_ratings, key=lambda x: x[2], reverse=True)[:top_n]
    return [camp for user, camp, rating in top_recommendations]

# 두 가지 방법 통합
def combined_recommendation(user_id, camp_name, top_n=5):
    # 코사인 유사도로 초기 후보 선정
    similar_camps = recommend_similar_camps(camp_name, top_n=20)
    
    # Surprise 모델을 사용하여 최종 추천
    user_ratings = [(user_id, camp, algo.predict(user_id, camp).est) for camp in similar_camps]
    top_recommendations = sorted(user_ratings, key=lambda x: x[2], reverse=True)[:top_n]
    return [camp for user, camp, rating in top_recommendations]

# 예시 통합 추천
print(combined_recommendation(1, '(주)아웃오브파크'))


['운일암반일암관광지 야영장', '졸드루오토캠핑장', '캠프 에이프릴', '오션포레글램핑', '블루마린 캠핑장']
